In [ ]:
from sqlalchemy.engine import create_engine
from sqlalchemy import inspect
from sqlalchemy import text
from pprint import pprint

import csv
import glob

from config import *


mmCourse = 'lgdm'

sqlite_engine = create_engine('sqlite:////Users/saisun/ProjectCode/uqx_tools/micromastersreports/databases/' + mmCourse + '_verified.db')
sqlite_connection = sqlite_engine.connect()

grade_file_path = '/Users/saisun/ProjectCode/uqx_tools/micromastersreports/data/' + mmCourse + '/'
grade_file_postfix = "_grade_report_*.csv"
profile_file_postfix = "_student_profile_info_*.csv"

micromasters_courses = mmCoursesMeta[mmCourse]

def db_refresh():
    sqlite_connection.execute("DELETE FROM learners")
    sqlite_connection.execute("DELETE FROM course_instancedetails")
    sqlite_connection.execute("DELETE FROM " + mmCourse + "_coursecompletion")
    
def db_insert (table, row):
    keys = row.keys()
    sql = 'INSERT INTO ' + table + ' ('
    sql = sql + ', '.join(keys)
    sql = sql + ') VALUES ('
    sql = sql + ', '.join([ ('"' + str(row[key]) + '"') for key in keys])
    sql = sql + ')'
    id = sqlite_connection.execute(text(sql))
    return id
    
def import_profile_fromcsv(course_prefix):
    profile_file_to_import = grade_file_path + course_prefix + '_*' + profile_file_postfix
    #print(profile_file_to_import, course_prefix)
    
    for file_name in glob.glob(profile_file_to_import):
        with open(file_name, mode='r', encoding='utf-8-sig') as infile:
            reader = csv.DictReader(infile)
            for row in reader:
#                pprint(row)
                student_id = int(row['id'])
                email = row['email']
                email = email.replace("'", "")
                username = row['username']
                name = row['name']
                name = name.replace(':', '')
                name = name.replace('?', '')
                name = name.replace('\'', '')
                name = name.replace("\"", "")
                country = row['country']
                insert_learner(student_id, email, username, name,  country)
                
def insert_learner(student_id, email, username, name='',  country=''):
    sql = "SELECT COUNT(*) FROM learners where student_id=%d;" % (student_id)
    result = sqlite_connection.execute(sql)
    no_students = 0
    for row in result:
        no_students = row[0]
    if no_students == 0:
        row = {'student_id': student_id, 'email':email, 'username':username, 'name': name, 'country': country}
        db_insert('learners', row)
        
    
def import_gradereport_fromcsv(course, course_prefix):
    file_to_import = grade_file_path + course_prefix  + '_*' + grade_file_postfix
    #print(file_to_import, course, course_prefix)

    for file_name in glob.glob(file_to_import):
        course_instance_start = int(file_name.find(course_prefix))
        course_instance_end = file_name.find('_grade_report')
        course_instance = file_name[course_instance_start:course_instance_end]
        print(course_instance)
        series = course_instance[len(course_prefix)+1:]
        split_series = series.split("T")
        term = split_series[0]
        year = split_series[1]
        #print(series, term, year)
                
        with open(file_name, mode='r', encoding='utf-8-sig') as infile:
            reader = csv.DictReader(infile)
            for row in reader:
                student_id = int(row['Student ID'])
                email = row['Email']
                email = email.replace("'", "")
                username = row['Username']   
                insert_learner(student_id, email, username)
                #insert course instance details
                grade = row['Grade']
                enrollment_track = row['Enrollment Track']
                verification_status = row['Verification Status']
                certificate_eligible = row['Certificate Eligible']
                certificate_delivered = row['Certificate Delivered']
                row = {
                    'learner_id': student_id,
                    'grade': grade,
                    'enrollment_track': enrollment_track,
                    'verification_status': verification_status,
                    'certificate_eligible': certificate_eligible,
                    'certificate_delivered': certificate_delivered,
                    'course': course,
                    'course_instance': course_instance,
                    'term': term,
                    'year': year
                }
                db_insert('course_instancedetails', row)
        
        
def export_micromasters_completionreport():
    print("export_micromasters_completionreport")
    sql = "SELECT * FROM learners;"
    result = sqlite_connection.execute(sql)
    for row in result:
        learner_dict = {
            'student_id': row[1],
            'email': row[2],
            'name': row[3],
            'username': row[4],
            'country': row[5]
        }
        no_certificate_eligible = 0
        
        for course, course_dict in micromasters_courses.items():
            grade_certificate = course_dict['grade']
            learner_dict[course + "_enrollment_track"] = "-"
            learner_dict[course + "_passed"] = "-"
            
            ci_certificate_sql = "SELECT * FROM course_instancedetails WHERE course='%s' AND learner_id=%d AND certificate_eligible = 'Y';" % (course, learner_dict['student_id'])            
            ci_certificate_sql_result = sqlite_connection.execute(ci_certificate_sql).fetchone()           
            if ci_certificate_sql_result is not None:
                no_certificate_eligible += 1
                learner_dict[course + "_enrollment_track"] = ci_certificate_sql_result[3]
                learner_dict[course + "_passed"] = 'Y'
            else:
                ci_sql = "SELECT * FROM course_instancedetails WHERE course='%s' AND learner_id=%d ORDER BY year DESC, term DESC;" % (course, learner_dict['student_id'])
                ci_query_result = sqlite_connection.execute(ci_sql).fetchone()
                if ci_query_result is not None:
                    learner_dict[course + "_enrollment_track"] = ci_query_result[3]
                    grade = ci_query_result[2]
                    grade_pass = 'N'
                    if grade >= grade_certificate:
                        grade_pass = 'Y'
                    learner_dict[course + "_passed"] = grade_pass
#                    if (ci_query_result[3] == 'audit') and (grade_pass == 'Y'):
#                        update_ci_id = int(ci_query_result[0])
#                        update_ci = "UPDATE course_instancedetails SET audit_passed='%s' WHERE id=%d; " %(grade_pass, update_ci_id)
#                        sqlite_connection.execute(update_ci)
                    
        learner_dict['no_certificate_eligible'] = no_certificate_eligible
        db_insert(mmCourse + '_coursecompletion', learner_dict)
                
def micromasters_completionreport_count_auditpassed():        
    print("micromasters_completionreport_count_auditpassed")
    sql = "SELECT * FROM " + mmCourse + "_coursecompletion;"
    result = sqlite_connection.execute(sql)
    for row in result:
        #pprint(row)
        update_id = int(row[0])
        no_auditedpassed_courses = 0
        if (row[5] == 'audit') and (row[6] == 'Y'):
            no_auditedpassed_courses +=1
        if (row[7] == 'audit') and (row[8] == 'Y'):
            no_auditedpassed_courses +=1
        if (row[9] == 'audit') and (row[10] == 'Y'):
            no_auditedpassed_courses +=1
        if (row[11] == 'audit') and (row[12] == 'Y'):
            no_auditedpassed_courses +=1
        
        updatesql = "UPDATE " + mmCourse + "_coursecompletion SET no_audited_passed=%d WHERE id=%d;" % (no_auditedpassed_courses, update_id)
        sqlite_connection.execute(updatesql)

# update the column 'audit_passed' in the table {course_instancedetails}  
def update_course_instancedetails_auditpassed():
    print('update_course_instancedetails_auditpassed')
    ci_query = 'SELECT * FROM course_instancedetails'
    result = sqlite_connection.execute(ci_query)
    for row in result:
        update_id = int(row[0])
        course = row[6]
        pass_line = micromasters_courses[course]['grade']
        #print('pass_line', pass_line)
        grade = row[2]
        enrollment_track = row[3]
        update_ci = "UPDATE course_instancedetails SET audit_passed='%s' WHERE id=%d; " %('N', update_id)
        if (enrollment_track == 'audit') and (grade >= pass_line):
            update_ci = "UPDATE course_instancedetails SET audit_passed='%s' WHERE id=%d; " %('Y', update_id)
        sqlite_connection.execute(update_ci)
        
        
        
db_refresh()
print('Import profile')
for course, course_dict in micromasters_courses.items():
    import_profile_fromcsv(course_dict['title'])

print('Import grade report')
for course, course_dict in micromasters_courses.items():
    import_gradereport_fromcsv(course, course_dict['title'])
    
export_micromasters_completionreport()
micromasters_completionreport_count_auditpassed()
update_course_instancedetails_auditpassed()

print('The End.')



